In [2]:
import Loader
from PreProcessor import Preprocessor
from LdaMalletHandler import LdaMalletHandler
from DocumentRetriever import DocumentRetriever
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers import LdaMallet
import pandas as pd

In [3]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;

### Load corpus

In [4]:
corpus = Loader.load_from_folder("data/Folhacompleto/")

In [5]:
len(corpus['articles'])

103913

In [6]:
corpus.keys()

dict_keys(['articles', 'dates'])

In [7]:
df = pd.DataFrame.from_dict(corpus)
df = df.drop('dates', 1)

### Pre-process and tokenize

In [8]:
p = Preprocessor(lang='portuguese')  
pre_processed = p.transform(corpus['articles'])

# NMF

In [9]:
# colocando todos os arquivos do dataset preprocessado na lista 
# e juntando as palavras, montando o artigo pós pré processamento
dataNMF = [' '.join(text) for text in pre_processed]

In [10]:
# criando o vectorizer e contando a frequencia das palavras
vectorizer = CountVectorizer(analyzer='word', max_features=500000) # pegando as 500000 melhores features
xCounts = vectorizer.fit_transform(dataNMF)

In [11]:
# aplicando o tfidf
transformer = TfidfTransformer(smooth_idf=False)
xTfidf = transformer.fit_transform(xCounts)

In [12]:
# fazendo a normalização
xTfidfNorm = normalize(xTfidf, norm='l1', axis=1)

In [13]:
# definindo o número de tópicos
# coloquei 50 por conta de ser o mesmo númeor que estávamos usando no LDA
nTopics = 200

In [14]:
modelNMF = NMF(n_components= nTopics, init='nndsvd')

In [15]:
# treinando o modelo
modelNMF.fit(xTfidfNorm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=200, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [16]:
def getTopicsNMF(model, nTopWords):
    '''funcao para pegar os tópicos da NMF
    model = modelo do NMF
    nTopWord = numero de top words'''
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(nTopics):
        words_ids = model.components_[i].argsort()[:-nTopWords - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    
    return word_dict

In [17]:
pd.DataFrame(getTopicsNMF(modelNMF, 10))

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 191,Topic # 192,Topic # 193,Topic # 194,Topic # 195,Topic # 196,Topic # 197,Topic # 198,Topic # 199,Topic # 200
0,fazenda,mes,cesta,congresso,dolar,policia,us,candidato,unido,indice,...,manaus,norte,agua,outubro,pessoas,emprego,anos,setor,queda,hotel
1,dallari,anterior,basica,revisao,fechou,delegado,bilhao,pdt,reino,igp,...,amazonas,coreia,sabesp,novembro,cerca,nivel,dois,autopecas,caiu,restaurante
2,ministerio,passado,custo,jobim,bolsas,crime,valor,psdb,irlanda,ipc,...,belem,americano,rodizio,setembro,matou,trabalho,tres,empregados,reducao,hoteis
3,secretario,comparacao,procon,parlamentares,mercado,civil,faturamento,brizola,britanico,consumidor,...,agencia,americana,abastecimento,relacao,menos,desemprego,quatro,privado,recuo,restaurantes
4,malan,relacao,dieese,constitucional,ouro,prisao,trimestre,presidencia,diana,variacao,...,ibama,kim,chuvas,dezembro,morreram,trabalhadores,idade,sinicesp,registrou,pacote
5,milton,recuo,higiene,partidos,indice,preso,custa,candidatura,londres,fgv,...,amazonia,sul,racionamento,passado,numero,postos,cinco,faturamento,supermercados,turismo
6,brasilia,atingiu,produtos,reformas,compra,acusado,milhao,apoio,major,ipa,...,am,americanos,bairros,desde,casas,empregos,ultimos,pesada,cairam,custa
7,assessor,igual,medio,reforma,venda,pf,internacionais,turno,ira,fundacao,...,acre,coreano,saneamento,subiu,fisicas,crescimento,meses,associacao,retracao,pratos
8,economica,bilhao,limpeza,emendas,exterior,delegacia,bi,tucano,princesa,percentual,...,segundo,il,zona,caiu,maioria,vagas,dez,sindipecas,relacao,praia
9,pedro,comparados,convenio,revisor,bm,prendeu,anunciou,amin,john,atacado,...,tocantins,nuclear,moradores,meses,manifestantes,fiesp,prisao,trabalhadores,quedas,hospedagem


In [18]:
topics = getTopicsNMF(modelNMF, 10)

In [19]:
file = open("results/FolhaCompletoNMF_{}t.txt".format(nTopics), "w")
for t in topics.keys():
    file.write(" ".join(topics[t])+"\n")
file.close() 